In [1]:
from typing import *
from yspecies import *
from yspecies.enums import *
from yspecies.dataset import *
from yspecies.misc import *

In [2]:
from dataclasses import dataclass
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

In [3]:
#settings
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)
pd.set_option('display.float_format', lambda x: '%.3f' % x)
import pprint
pp = pprint.PrettyPrinter(indent=4)

In [4]:
from pathlib import Path
locations: Locations = Locations("./") if Path("./data").exists() else Locations("../")

In [5]:
external = shap_results = locations.output.external 
intersections = locations.output.intersections
interim = locations.interim

In [6]:
species = pd.read_csv(interim.dir / "selected_species.tsv", sep="\t")[["species", "common_name", "lifespan"]]
species.head(10)

,species,common_name,lifespan
0,Ailuropoda_melanoleuca,Giant panda,36.800
1,Aotus_nancymaae,Nancy Ma's night monkey,20.000
2,Bos_taurus,Domestic cattle,20.000
3,Callithrix_jacchus,White-tufted-ear marmoset,22.800
4,Canis_lupus_familiaris,Domestic dog,24.000
5,Capra_hircus,Domestic goat,20.800
6,Cavia_aperea,Brazilian guinea pig,6.000
7,Cavia_porcellus,Guinea pig,12.000
8,Equus_caballus,Horse,57.000
9,Felis_catus,Domestic cat,30.000


In [12]:
i_pro= pd.read_csv(intersections / "pro_intersections_5_and_7.tsv",sep="\t").sort_values("MLS_kendall_tau", ascending=False)[["reference_gene", "symbol"]].rename(columns={"reference_gene": "Homo_sapiens"}).set_index("Homo_sapiens")
i_anti= pd.read_csv(intersections / "anti_intersections_5_and_7.tsv",sep="\t").sort_values("MLS_kendall_tau", ascending=True)[["reference_gene", "symbol"]].rename(columns={"reference_gene": "Homo_sapiens"}).set_index("Homo_sapiens")
i_anti.head(10)

,symbol
Homo_sapiens,
ENSG00000107815,TWNK
ENSG00000089234,BRAP
ENSG00000198663,C6orf89
ENSG00000214827,MTCP1
ENSG00000149577,SIDT2
ENSG00000152580,IGSF10
ENSG00000164879,CA3
ENSG00000121897,LIAS
ENSG00000204231,RXRB


In [8]:
pro = pd.read_csv(external.dir / "for_proteins" / "pro.tsv",sep="\t")
anti = pd.read_csv(external.dir / "for_proteins" / "anti.tsv",sep="\t")
pro.head(10)

,Homo_sapiens,Loxodonta_africana,Gorilla_gorilla,Pan_troglodytes,Pongo_pygmaeus,Equus_caballus,Pan_paniscus,Tursiops_truncatus,Macaca_mulatta,Macaca_fascicularis,Macaca_nemestrina,Ailuropoda_melanoleuca,Ursus_americanus,Heterocephalus_glaber,Rhinolophus_ferrumequinum,Vombatus_ursinus,Felis_catus,Sus_scrofa,Lynx_canadensis,Bos_grunniens,Canis_lupus_familiaris,Rhinopithecus_bieti,Ovis_aries,Callithrix_jacchus,Ornithorhynchus_anatinus,Dasypus_novemcinctus,Phascolarctos_cinereus,Capra_hircus,Suricata_suricatta,Bos_taurus,Aotus_nancymaae,Otolemur_garnettii,Microcebus_murinus,Chinchilla_lanigera,Sarcophilus_harrisii,Cavia_porcellus,Erinaceus_europaeus,Tupaia_belangeri,Oryctolagus_cuniculus,Ictidomys_tridecemlineatus,Meriones_unguiculatus,Cavia_aperea,Monodelphis_domestica,Mus_musculus,Mesocricetus_auratus,Rattus_norvegicus,Mus_spicilegus,Mus_caroli
0,ENSG00000133256,ENSLAFG00000009245,ENSGGOG00000005730,ENSPTRG00000046026,ENSPPYG00000014504,ENSECAG00000017337,ENSPPAG00000037791,ENSTTRG00000002928,ENSMMUG00000003282,ENSMFAG00000039102,ENSMNEG00000045444,ENSAMEG00000016646,ENSUAMG00000015877,ENSHGLG00100013558,ENSRFEG00010013462,ENSVURG00010014792;ENSVURG00010019541,ENSFCAG00000008224,ENSSSCG00000036645,ENSLCNG00005001203,ENSBGRG00000020982,ENSCAFG00030021135;ENSCAFG00000016782;ENSCAFG0...,ENSRBIG00000034185,ENSOARG00000016949,ENSCJAG00000001990,ENSOANG00000006518,ENSDNOG00000002553,ENSPCIG00000014391,ENSCHIG00000020394,ENSSSUG00005006261,ENSBTAG00000017480,ENSANAG00000029619,ENSOGAG00000012260,ENSMICG00000010725,ENSCLAG00000012451,ENSSHAG00000012453,ENSCPOG00000022569,ENSEEUG00000009058,ENSTBEG00000005887,ENSOCUG00000038876,ENSSTOG00000020412,ENSMUGG00000019544,ENSCAPG00000012847,ENSMODG00000005968,ENSMUSG00000029491,ENSMAUG00000018594,ENSRNOG00000000065,ENSMSIG00000020631,MGP_CAROLIEiJ_G0027518
1,ENSG00000160323,ENSLAFG00000011278,ENSGGOG00000012541,ENSPTRG00000022942,ENSPPYG00000019736,ENSECAG00000008699,ENSPPAG00000030531,ENSTTRG00000007685,ENSMMUG00000000937,ENSMFAG00000000040,ENSMNEG00000034336,ENSAMEG00000011605,ENSUAMG00000018673,ENSHGLG00100015138,ENSRFEG00010016293,ENSVURG00010012105,ENSFCAG00000026549,ENSSSCG00000021241,ENSLCNG00005021964,ENSBGRG00000003687,ENSCAFG00000019776;ENSCAFG00040001104;ENSCAFG0...,ENSRBIG00000040768,ENSOARG00000003336,ENSCJAG00000009791,ENSOANG00000002455,ENSDNOG00000048133,ENSPCIG00000017381,ENSCHIG00000025823,ENSSSUG00005023329,ENSBTAG00000015093,ENSANAG00000032285,ENSOGAG00000014043,ENSMICG00000011799,ENSCLAG00000007542,ENSSHAG00000006104,ENSCPOG00000010285,ENSEEUG00000012163,ENSTBEG00000004466,ENSOCUG00000039392,ENSSTOG00000003765,ENSMUGG00000004109,ENSCAPG00000007044,ENSMODG00000039571,ENSMUSG00000014852,ENSMAUG00000002369,ENSRNOG00000005780,ENSMSIG00000019355,MGP_CAROLIEiJ_G0023333
2,ENSG00000129187,ENSLAFG00000018439,ENSGGOG00000016412,ENSPTRG00000016624,ENSPPYG00000015223,ENSECAG00000017516;ENSECAG00000000947,ENSPPAG00000022425,ENSTTRG00000001460,ENSMMUG00000000284,ENSMFAG00000034653,ENSMNEG00000033220,ENSAMEG00000008118,ENSUAMG00000013234,ENSHGLG00100018129,ENSRFEG00010017093,ENSVURG00010007106,ENSFCAG00000044388,ENSSSCG00000015775,ENSLCNG00005001180,ENSBGRG00000007650,ENSCAFG00000031175;ENSCAFG00030011853;ENSCAFG0...,ENSRBIG00000044833,ENSOARG00000006992,ENSCJAG00000005526,ENSOANG00000006238,ENSDNOG00000000721,ENSPCIG00000013744,ENSCHIG00000010700,ENSSSUG00005000048,ENSBTAG00000020282,ENSANAG00000026089,ENSOGAG00000000583,ENSMICG00000006929,ENSCLAG00000012757,ENSSHAG00000015218;ENSSHAG00000003799,ENSCPOG00000040207,ENSEEUG00000006654,ENSTBEG00000015910,ENSOCUG00000004708,ENSSTOG00000005462,ENSMUGG00000025677,ENSCAPG00000005870,ENSMODG00000004124,ENSMUSG00000031562,ENSMAUG00000015286,ENSRNOG00000013215,ENSMSIG00000030915,MGP_CAROLIEiJ_G0031048
3,ENSG00000069275,ENSLAFG00000021663,ENSGGOG00000039124,ENSPTRG00000048501,ENSPPYG00000000299,ENSECAG00000010336,ENSPPAG00000031082,ENSTTRG00000010469,ENSMMUG00000055835,ENSMFAG00000041134,ENSMNEG00000012609,ENSAMEG00000012950,ENSUAMG00000022802,ENSH

In [14]:
pro = pd.read_csv("/data/species/pro.tsv",sep="\t")
anti = pd.read_csv("/data/species/anti.tsv",sep="\t")
pro_t = i_pro.reset_index().merge(pro, on="Homo_sapiens").set_index("symbol").T.reset_index().rename(columns={"index": "species"})
anti_t = i_anti.reset_index().merge(anti, on="Homo_sapiens").set_index("symbol").T.reset_index().rename(columns={"index": "species"})
anti_t

symbol,species,TWNK,BRAP,C6orf89,MTCP1,SIDT2,IGSF10,CA3,LIAS,RXRB,RWDD1,SMPDL3A,ACTC1,COX5B,CDADC1,CTPS2,ADPRM,ABHD16A
0,Homo_sapiens,ENSG00000107815,ENSG00000089234,ENSG00000198663,ENSG00000214827,ENSG00000149577,ENSG00000152580,ENSG00000164879,ENSG00000121897,ENSG00000204231,ENSG00000111832,ENSG00000172594,ENSG00000159251,ENSG00000135940,ENSG00000102543,ENSG00000047230,ENSG00000170222,ENSG00000204427
1,Loxodonta_africana,ENSLAFG00000015859,ENSLAFG00000006701,ENSLAFG00000000208,ENSLAFG00000028512,ENSLAFG00000030954,ENSLAFG00000000038,ENSLAFG00000004404,ENSLAFG00000012023,ENSLAFG00000002141,ENSLAFG00000027334,ENSLAFG00000010744,ENSLAFG00000021518,ENSLAFG00000009386,ENSLAFG00000012861,ENSLAFG00000013533,ENSLAFG00000008416,ENSLAFG00000009495
2,Gorilla_gorilla,ENSGGOG00000015421,ENSGGOG00000007087,ENSGGOG00000013941,ENSGGOG00000037091,ENSGGOG00000014318,ENSGGOG00000002875,ENSGGOG00000010673,ENSGGOG00000014977,ENSGGOG00000009823,ENSGGOG00000026889,ENSGGOG00000011523,ENSGGOG00000023984,ENSGGOG00000001142,ENSGGOG00000005192,ENSGGOG00000006951,ENSGGOG00000009051,ENSGGOG00000009970
3,Pan_troglodytes,ENSPTRG00000029751,ENSPTRG00000005461,ENSPTRG00000024272,ENSPTRG00000022457,ENSPTRG00000004319,ENSPTRG00000015539,ENSPTRG00000052102,ENSPTRG00000015994,ENSPTRG00000018034,ENSPTRG00000018534,ENSPTRG00000018565,ENSPTRG00000006890,ENSPTRG00000012257;ENSPTRG00000048245,ENSPTRG00000005871,ENSPTRG00000045235;ENSPTRG00000021698,ENSPTRG00000008775,ENSPTRG00000017977
4,Pongo_pygmaeus,ENSPPYG00000002576,ENSPPYG00000004964,ENSPPYG00000016555,ENSPPYG00000020898,ENSPPYG00000003908,ENSPPYG00000014216,ENSPPYG00000018717,ENSPPYG00000014676,ENSPPYG00000017239,ENSPPYG00000016953,ENSPPYG00000016980,ENSPPYG00000006318,ENSPPYG00000012078,ENSPPYG00000005365,ENSPPYG00000020154,ENSPPYG00000007985,ENSPPYG00000016450
5,Equus_caballus,ENSECAG00000017734,ENSECAG00000007549,ENSECAG00000024122,ENSECAG00000043072,ENSECAG00000021466,ENSECAG00000023161,ENSECAG00000016228,ENSECAG00000015796,ENSECAG00000023041,ENSECAG00000006587,ENSECAG00000023408,ENSECAG00000013022,ENSECAG00000007968,ENSECAG00000020688,ENSECAG00000017669,ENSECAG00000025154,ENSECAG00000013375
6,Pan_paniscus,ENSPPAG00000035055,ENSPPAG00000042331,ENSPPAG00000042318,ENSPPAG00000029273,ENSPPAG00000029761,ENSPPAG00000033391,ENSPPAG00000043034,ENSPPAG00000040519,ENSPPAG00000038590,ENSPPAG00000035047,ENSPPAG00000013616,ENSPPAG00000029795,ENSPPAG00000041629,ENSPPAG00000028450,ENSPPAG00000031030,ENSPPAG00000043053,ENSPPAG00000040009
7,Tursiops_truncatus,ENSTTRG00000015112,ENSTTRG00000010873,ENSTTRG00000002299,ENSTTRG00000008159,ENSTTRG00000000973,ENSTTRG00000003206,ENSTTRG00000005803,ENSTTRG00000014293,ENSTTRG00000004291,ENSTTRG00000016549,ENSTTRG00000000252,ENSTTRG00000014799,ENSTTRG00000013490,ENSTTRG00000004052,ENSTTRG00000013328,ENSTTRG00000000069,ENSTTRG00000014105
8,Macaca_mulatta,ENSMMUG00000064033,ENSMMUG00000017107,ENSMMUG00000005004,ENSMMUG00000010251,ENSMMUG00000002053,ENSMMUG00000001107,ENSMMUG00000014890,ENSMMUG00000013851,ENSMMUG00000003858,ENSMMUG00000007415,ENSMMUG00000054098,ENSMMUG00000007027,ENSMMUG00000050877,ENSMMUG00000013470,ENSMMUG00000004797,ENSMMUG00000057593,ENSMMUG00000015686
9,Macaca_fascicularis,ENSMFAG00000032824,ENSMFAG00000036173,ENSMFAG00000042966,ENSMFAG00000001317,ENSMFAG00000032882,ENSMFAG00000002055,ENSMFAG00000000412,ENSMFAG00000040861,ENSMFAG00000003583,ENSMFAG00000042598,ENSMFAG00000043472,ENSMFAG00000039604,ENSMFAG00000034597;ENSMFAG00000036411;ENSMFAG0...,ENSMFAG00000032976,ENSMFAG00000009688,ENSMFAG00000042103,ENSMFAG00000000198


In [10]:
pro_genes = species.sort_values("lifespan", ascending=False).merge(pro_t.rename(columns={"symbol": "species"}),on="species", how="outer").set_index("species")
pro_genes

,common_name,lifespan,METTL5,TRIM66,ADAMTS13,DCTD,NUCKS1,SPATA20,ARMC12,CLDN16,NAALADL1,RASSF4,PDE6B,MYOM1,TMEM182,NSMCE1,POLE,UPP1,SUMO1,CPVL,MOV10L1,MBD4,AC020929.1
species,,,,,,,,,,,,,,,,,,,,,,,
Homo_sapiens,Human,122.500,ENSG00000138382,ENSG00000166436,ENSG00000160323,ENSG00000129187,ENSG00000069275,ENSG00000006282,ENSG00000157343,ENSG00000113946,ENSG00000168060,ENSG00000107551,ENSG00000133256,ENSG00000101605,ENSG00000170417,ENSG00000169189,ENSG00000177084,ENSG00000183696,ENSG00000116030,ENSG00000106066,ENSG00000073146,ENSG00000129071,ENSG00000160460
Loxodonta_africana,African bush elephant,65.000,ENSLAFG00000018044,ENSLAFG00000000423,ENSLAFG00000011278,ENSLAFG00000018439,ENSLAFG00000021663,ENSLAFG00000022872,ENSLAFG00000032578,ENSLAFG00000008037,ENSLAFG00000001142,ENSLAFG00000012041,ENSLAFG00000009245,ENSLAFG00000014655,ENSLAFG00000011215,ENSLAFG00000018542,ENSLAFG00000010802,ENSLAFG00000000594,ENSLAFG00000022855,ENSLAFG00000002811,ENSLAFG00000014159,ENSLAFG00000010609,ENSLAFG00000020815
Gorilla_gorilla,Gorilla,60.100,ENSGGOG00000022484,ENSGGOG00000016600,ENSGGOG00000012541,ENSGGOG00000016412,ENSGGOG00000039124,ENSGGOG00000003379,ENSGGOG00000015515,ENSGGOG00000025366,ENSGGOG00000002695,ENSGGOG00000012833,ENSGGOG00000005730,ENSGGOG00000044057,ENSGGOG00000037042,ENSGGOG00000006501,ENSGGOG00000010005,ENSGGOG00000028074,ENSGGOG00000026439,ENSGGOG00000009483,ENSGGOG00000010110,ENSGGOG00000010698,ENSGGOG00000004627
Pan_troglodytes,Chimpanzee,59.400,ENSPTRG00000012621,ENSPTRG00000050097,ENSPTRG00000022942,ENSPTRG00000016624,ENSPTRG00000048501,ENSPTRG00000009404,ENSPTRG00000018089,ENSPTRG00000015739,ENSPTRG00000003859,ENSPTRG00000002452,ENSPTRG00000046026,ENSPTRG00000009834,ENSPTRG00000012308,ENSPTRG00000007912,ENSPTRG00000045365,ENSPTRG00000045643;ENSPTRG00000019177,ENSPTRG00000012817,ENSPTRG00000019029,ENSPTRG00000044273,ENSPTRG00000015377,ENSPTRG00000011000
Equus_caballus,Horse,57.000,ENSECAG00000004756,ENSECAG00000021487,ENSECAG00000008699,ENSECAG00000017516;ENSECAG00000000947,ENSECAG00000010336,ENSECAG00000008281,ENSECAG00000011493,ENSECAG00000021850,ENSECAG00000018798,ENSECAG00000007066,ENSECAG00000017337,ENSECAG00000013468,ENSECAG00000028489,ENSECAG00000024006,ENSECAG00000001653,ENSECAG00000008193,ENSECAG00000024693,ENSECAG00000024795,ENSECAG00000011063,ENSECAG00000022106,ENSECAG00000042883
Pan_paniscus,Pygmy chimpanzee or bonobo,55.000,ENSPPAG00000031600,ENSPPAG00000037497,ENSPPAG00000030531,ENSPPAG00000022425,ENSPPAG00000031082,ENSPPAG00000031622,ENSPPAG00000005870,ENSPPAG00000031653,ENSPPAG00000043800,ENSPPAG00000029784,ENSPPAG00000037791,ENSPPAG00000037926,ENSPPAG00000038490,ENSPPAG00000043297,ENSPPAG00000034043,ENSPPAG00000034370,ENSPPAG00000039520,ENSPPAG00000039137,ENSPPAG00000029459,ENSPPAG00000036528,ENSPPAG00000042534
Tursiops_truncatus,Bottlenosed dolphin,51.600,ENSTTRG00000002154,ENSTTRG00000004599,ENSTTRG00000007685,ENSTTRG00000001460,ENSTTRG00000010469,ENSTTRG00000000215,ENSTTRG00000014321,ENSTTRG00000011535,ENSTTRG00000000326,ENSTTRG00000011719,ENSTTRG00000002928,ENSTTRG00000008310,ENSTTRG00000011185,ENSTTRG00000008983,ENSTTRG00000010629,ENSTTRG00000012047,ENSTTRG00000000642,ENSTTRG00000014785,ENSTTRG00000000879,ENSTTRG00000011265,ENSTTRG00000007143
Macaca_mulatta,Rhesus monkey,40.000,ENSMMUG00000023054,ENSMMUG00000021827,ENSMMUG00000000937,ENSMMUG00000000284,ENSMMUG00000055835,ENSMMUG00000017252,ENSMMUG00000059809,ENSMMUG00000018671,ENSMMUG00000012315,ENSMMUG00000017489,ENSMMUG00000003282,ENSMMUG00000019892,ENSMMUG00000004169,ENSMMUG00000015832,ENSMMUG00000015463,ENSMMUG00000007037,ENSMMUG00000005240,ENSMMUG00000020184,ENSMMUG00000009120,ENSMMUG00000012723,ENSMMUG00000020796;ENSMMUG00000051535
Macaca_fascicularis,Long-tailed macaque,39.000,ENSMFAG00000046280,ENSMFAG00000002122,ENSMFAG00000000040,ENSMFAG00000034653,ENSMFAG00000041134,ENSMFAG00000044525,ENSMFAG00000038125,ENSMFAG00000043262,ENSMFAG00000035435,ENSMFAG00000031803,ENSMFAG00000039102,ENSMFAG00000041420,ENSMFAG00000036897,ENSMFAG00000041758,ENSMFA

In [15]:
anti_genes = species.sort_values("lifespan", ascending=False).merge(anti_t.rename(columns={"symbol": "species"}),on="species", how="outer").set_index("species")
anti_genes

,common_name,lifespan,TWNK,BRAP,C6orf89,MTCP1,SIDT2,IGSF10,CA3,LIAS,RXRB,RWDD1,SMPDL3A,ACTC1,COX5B,CDADC1,CTPS2,ADPRM,ABHD16A
species,,,,,,,,,,,,,,,,,,,
Homo_sapiens,Human,122.500,ENSG00000107815,ENSG00000089234,ENSG00000198663,ENSG00000214827,ENSG00000149577,ENSG00000152580,ENSG00000164879,ENSG00000121897,ENSG00000204231,ENSG00000111832,ENSG00000172594,ENSG00000159251,ENSG00000135940,ENSG00000102543,ENSG00000047230,ENSG00000170222,ENSG00000204427
Loxodonta_africana,African bush elephant,65.000,ENSLAFG00000015859,ENSLAFG00000006701,ENSLAFG00000000208,ENSLAFG00000028512,ENSLAFG00000030954,ENSLAFG00000000038,ENSLAFG00000004404,ENSLAFG00000012023,ENSLAFG00000002141,ENSLAFG00000027334,ENSLAFG00000010744,ENSLAFG00000021518,ENSLAFG00000009386,ENSLAFG00000012861,ENSLAFG00000013533,ENSLAFG00000008416,ENSLAFG00000009495
Gorilla_gorilla,Gorilla,60.100,ENSGGOG00000015421,ENSGGOG00000007087,ENSGGOG00000013941,ENSGGOG00000037091,ENSGGOG00000014318,ENSGGOG00000002875,ENSGGOG00000010673,ENSGGOG00000014977,ENSGGOG00000009823,ENSGGOG00000026889,ENSGGOG00000011523,ENSGGOG00000023984,ENSGGOG00000001142,ENSGGOG00000005192,ENSGGOG00000006951,ENSGGOG00000009051,ENSGGOG00000009970
Pan_troglodytes,Chimpanzee,59.400,ENSPTRG00000029751,ENSPTRG00000005461,ENSPTRG00000024272,ENSPTRG00000022457,ENSPTRG00000004319,ENSPTRG00000015539,ENSPTRG00000052102,ENSPTRG00000015994,ENSPTRG00000018034,ENSPTRG00000018534,ENSPTRG00000018565,ENSPTRG00000006890,ENSPTRG00000012257;ENSPTRG00000048245,ENSPTRG00000005871,ENSPTRG00000045235;ENSPTRG00000021698,ENSPTRG00000008775,ENSPTRG00000017977
Equus_caballus,Horse,57.000,ENSECAG00000017734,ENSECAG00000007549,ENSECAG00000024122,ENSECAG00000043072,ENSECAG00000021466,ENSECAG00000023161,ENSECAG00000016228,ENSECAG00000015796,ENSECAG00000023041,ENSECAG00000006587,ENSECAG00000023408,ENSECAG00000013022,ENSECAG00000007968,ENSECAG00000020688,ENSECAG00000017669,ENSECAG00000025154,ENSECAG00000013375
Pan_paniscus,Pygmy chimpanzee or bonobo,55.000,ENSPPAG00000035055,ENSPPAG00000042331,ENSPPAG00000042318,ENSPPAG00000029273,ENSPPAG00000029761,ENSPPAG00000033391,ENSPPAG00000043034,ENSPPAG00000040519,ENSPPAG00000038590,ENSPPAG00000035047,ENSPPAG00000013616,ENSPPAG00000029795,ENSPPAG00000041629,ENSPPAG00000028450,ENSPPAG00000031030,ENSPPAG00000043053,ENSPPAG00000040009
Tursiops_truncatus,Bottlenosed dolphin,51.600,ENSTTRG00000015112,ENSTTRG00000010873,ENSTTRG00000002299,ENSTTRG00000008159,ENSTTRG00000000973,ENSTTRG00000003206,ENSTTRG00000005803,ENSTTRG00000014293,ENSTTRG00000004291,ENSTTRG00000016549,ENSTTRG00000000252,ENSTTRG00000014799,ENSTTRG00000013490,ENSTTRG00000004052,ENSTTRG00000013328,ENSTTRG00000000069,ENSTTRG00000014105
Macaca_mulatta,Rhesus monkey,40.000,ENSMMUG00000064033,ENSMMUG00000017107,ENSMMUG00000005004,ENSMMUG00000010251,ENSMMUG00000002053,ENSMMUG00000001107,ENSMMUG00000014890,ENSMMUG00000013851,ENSMMUG00000003858,ENSMMUG00000007415,ENSMMUG00000054098,ENSMMUG00000007027,ENSMMUG00000050877,ENSMMUG00000013470,ENSMMUG00000004797,ENSMMUG00000057593,ENSMMUG00000015686
Macaca_fascicularis,Long-tailed macaque,39.000,ENSMFAG00000032824,ENSMFAG00000036173,ENSMFAG00000042966,ENSMFAG00000001317,ENSMFAG00000032882,ENSMFAG00000002055,ENSMFAG00000000412,ENSMFAG00000040861,ENSMFAG00000003583,ENSMFAG00000042598,ENSMFAG00000043472,ENSMFAG00000039604,ENSMFAG00000034597;ENSMFAG00000036411;ENSMFAG0...,ENSMFAG00000032976,ENSMFAG00000009688,ENSMFAG00000042103,ENSMFAG00000000198


In [79]:
pro_genes.to_csv("/data/species/pro_genes.tsv",sep="\t",  index=True)
anti_genes.to_csv("/data/species/anti_genes.tsv",sep="\t",  index=True)
